In [1]:
#Importing Libraries

import sympy as smp
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import solve_ivp

from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter

from IPython.display import HTML
import matplotlib as mpl


mpl.rcParams['animation.embed_limit'] = 50

mpl.rcParams['animation.ffmpeg_path'] = r'C:\ProgramData\chocolatey\bin\ffmpeg.exe'

plt.style.use('dark_background')

# One double pendulum system

In [2]:
#Defining variabels and constants
t=smp.symbols('t', real='true',positive='true')
m1 , m2 , l1 , l2 , g = smp.symbols('m_1 m_2 l_1 l_2 g' , real='true' , positive='true')

In [3]:
#Defining the angles
theta1 = smp.Function('theta_1 ')(t)
theta2 = smp.Function('theta_2')(t)

In [4]:
#Defining the cartesian coordinates
x1=l1*smp.sin(theta1)
y1=-l1*smp.cos(theta1)

x2=x1+l2*smp.sin(theta2)
y2=y1-l2*smp.cos(theta2)

In [5]:
#Computing the velocities in cartesian coordinates
vx1=smp.diff(x1,t)
vy1=smp.diff(y1,t)

vx2=smp.diff(x2,t)
vy2=smp.diff(y2,t)

In [6]:
#Computing the energies
T1=smp.Rational(1,2) * m1 * (vx1**2 + vy1**2).simplify()
T2=smp.Rational(1,2) * m2 * (vx2**2 + vy2**2).simplify()


U1=m1*g*y1.simplify()
U2=m2*g*y2.simplify()


T=T1+T2.simplify()
U=U1+U2.simplify()

In [7]:
#The Lagrangian
L=T-U.simplify()

In [8]:
#Solving Euler-Lagrange equations for the 2 angles
dld1=smp.diff(L,theta1)
ddt_dld1=smp.diff(smp.diff(L,smp.diff(theta1,t)),t)

E1=dld1-ddt_dld1.simplify()


dld2=smp.diff(L,theta2)
ddt_dld2=smp.diff(smp.diff(L,smp.diff(theta2,t)),t)

E2=dld2-ddt_dld2.simplify()

In [9]:
#Solving each DE for the acceleration
sols = smp.solve([E1,E2],[smp.diff(theta1,t,2),smp.diff(theta2,t,2)])

In [10]:
#Defining new names for the angles and angular velocities for convenience
u1, u2 , w1 , w2 = smp.symbols('u1 u2 w1 w2')

#Substituting the new names in the DEs
f_expr=sols[smp.diff(theta1,t,2)].subs({
    smp.diff(theta1,t): w1,
    smp.diff(theta2,t): w2,
    theta1:u1,
    theta2:u2
})


q_expr=sols[smp.diff(theta2,t,2)].subs({
    smp.diff(theta1,t): w1,
    smp.diff(theta2,t): w2,
    theta1:u1,
    theta2:u2
})

In [11]:
#Substituting values for the constants
m_1 , m_2 , l_1 , l_2 , grav= 0.15, 0.2, 0.4, 0.8, 9.81
params = {m1: m_1, m2: m_2, l1: l_1, l2: l_2, g: grav}

f_expr_num = f_expr.subs(params)
q_expr_num = q_expr.subs(params)

#Converting the symolic expression to a numerical one
f = smp.lambdify((u1, u2, w1 ,w2), f_expr_num, "numpy")
q = smp.lambdify((u1, u2, w1, w2), q_expr_num, "numpy")

In [12]:
# Right-hand side of the ODE system
def rhs(t,x):
    u1 , u2 , w1 , w2 = x
    du1_dt=w1
    du2_dt=w2
    dw1_dt=f(u1 , u2 , w1 , w2)
    dw2_dt=q(u1 , u2 , w1 , w2)
    return [du1_dt , du2_dt , dw1_dt , dw2_dt]

In [13]:
#Initial angles in rad
u10=np.pi
u20=np.pi/2
#Initial angular velocities in rad/s
w10=0
w20=0

#List of ICs for the solver
x0=[u10 , u20 , w10 , w20]

#Time span and dt
tsp=(0,20)
t_eval=np.linspace(tsp[0],tsp[1],1000)

In [14]:
#Solving the DE numerically using the DOP853 solver
sol = solve_ivp(rhs, tsp, x0, t_eval=t_eval, method='DOP853',rtol=1e-9, atol=1e-9)

In [15]:
#Extracting the array of values from the solver
u1 = sol.y[0]
u2 = sol.y[1]
w1 = sol.y[2]
w2 = sol.y[3]

In [ ]:
#Plotting angular positions (angles) Vs. time
plt.plot(t_eval,u1,label='bob 1',color='red')
plt.plot(t_eval,u2,label='bob 2')
plt.xlabel('Time')
plt.ylabel('Angular position')
plt.legend()
plt.grid()

In [17]:
#Converting to cartesian
x1=l_1*np.sin(u1)
y1=-l_1*np.cos(u1)

x2=x1+l_2*np.sin(u2)
y2=y1-l_2*np.cos(u2)

In [ ]:
#Animating the trajectory of the double pendulum

#Setting up the figure environment
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)


#Create line artists
line, = ax.plot([], [], 'o-', lw=2)
trace1, = ax.plot([], [], 'b-', lw=0.5)
trace2, = ax.plot([], [], 'r-', lw=0.5)


#Arrays for path of bobs
path1_x, path1_y = [], []
path2_x, path2_y = [], []



#Initialization function
def init():
    line.set_data([], [])
    trace1.set_data([], [])
    trace2.set_data([], [])
    return line, trace1, trace2

#Updating function
def update(frame):
    
    line.set_data([0, x1[frame], x2[frame]],
                  [0, y1[frame], y2[frame]])

    
    path1_x.append(x1[frame])
    path1_y.append(y1[frame])
    trace1.set_data(path1_x, path1_y)

    path2_x.append(x2[frame])
    path2_y.append(y2[frame])
    trace2.set_data(path2_x, path2_y)

    return line, trace1, trace2

#Animation
ani = FuncAnimation(fig, update, frames=len(u1),
                    init_func=init, blit=True, interval=20)
#Animation display using HTML5
HTML(ani.to_jshtml())

# One figure, Multiple pendulums

In [19]:
d_theta=0.000009948

#I angle of first pendulum in rad
P1_m1=np.pi/2
P1_m2=np.pi

#I angle of second pendulum in rad
P2_m1=(np.pi/2) +d_theta
P2_m2=np.pi +d_theta

#I angle of third pendulum in rad
P3_m1=(np.pi/2) +2*(d_theta)
P3_m2=np.pi +2*(d_theta)

#I angle of fourth pendulum in rad
P4_m1=(np.pi/2) +3*(d_theta)
P4_m2=np.pi+3*(d_theta)


#I angle of fifth pendulum in rad
P5_m1=(np.pi/2)+4*(d_theta)
P5_m2=np.pi+4*(d_theta)


#I velocity of first pendulum in rad/s
P1_v1=0
P1_v2=0

#I velocity of second pendulum in rad/s
P2_v1=0
P2_v2=0

#I velocity of third pendulum in rad/s
P3_v1=0
P3_v2=0

#I velocity of fourth pendulum in rad/s
P4_v1=0
P4_v2=0


#I velocity of fifth pendulum in rad/s
P5_v1=0
P5_v2=0



#ICs of each pendulum for the solver
x1_0=[P1_m1 , P1_m2 , P1_v1 , P1_v2]
x2_0=[P2_m1 , P2_m2 , P2_v1 , P2_v2]
x3_0=[P3_m1 , P3_m2 , P3_v1 , P3_v2]
x4_0=[P4_m1 , P4_m2 , P4_v1 , P4_v2]
x5_0=[P5_m1 , P5_m2 , P5_v1 , P5_v2]

In [20]:
#Solving the ODEs
sol_1 = solve_ivp(rhs, tsp, x1_0, t_eval=t_eval, method='DOP853',rtol=1e-9, atol=1e-9)

sol_2 = solve_ivp(rhs, tsp, x2_0, t_eval=t_eval, method='DOP853',rtol=1e-9, atol=1e-9)

sol_3 = solve_ivp(rhs, tsp, x3_0, t_eval=t_eval, method='DOP853',rtol=1e-9, atol=1e-9)

sol_4 = solve_ivp(rhs, tsp, x4_0, t_eval=t_eval, method='DOP853',rtol=1e-9, atol=1e-9)

sol_5 = solve_ivp(rhs, tsp, x5_0, t_eval=t_eval, method='DOP853',rtol=1e-9, atol=1e-9)

In [21]:
#Extracting values


#First pendulum
P1_theta1 = sol_1.y[0]
P1_theta2 = sol_1.y[1]
P1_omega1 = sol_1.y[2]
P1_omega2 = sol_1.y[3]


#second pendulum
P2_theta1 = sol_2.y[0]
P2_theta2 = sol_2.y[1]
P2_omega1 = sol_2.y[2]
P2_omega2 = sol_2.y[3]


#third pendulum
P3_theta1 = sol_3.y[0]
P3_theta2 = sol_3.y[1]
P3_omega1 = sol_3.y[2]
P3_omega2 = sol_3.y[3]


#fourth pendulum
P4_theta1 = sol_4.y[0]
P4_theta2 = sol_4.y[1]
P4_omega1 = sol_4.y[2]
P4_omega2 = sol_4.y[3]


#fifth pendulum
P5_theta1 = sol_5.y[0]
P5_theta2 = sol_5.y[1]
P5_omega1 = sol_5.y[2]
P5_omega2 = sol_5.y[3]

In [ ]:
#Plotting angles Vs. time

#First pendulum
plt.plot(t_eval,P1_theta1,label='Pendulum 1 bob 1')
plt.plot(t_eval,P1_theta2,label='Pendulum 1 bob 2')

#Second pendulum
plt.plot(t_eval,P2_theta1,label='Pendulum 2 bob 1')
plt.plot(t_eval,P2_theta2,label='Pendulum 2 bob 2')

#Third pendulum
plt.plot(t_eval,P3_theta1,label='Pendulum 3 bob 1')
plt.plot(t_eval,P3_theta2,label='Pendulum 3 bob 2')

#Fourth pendulum
plt.plot(t_eval,P4_theta1,label='Pendulum 4 bob 1')
plt.plot(t_eval,P4_theta2,label='Pendulum 4 bob 2')

#Fifth pendulum
plt.plot(t_eval,P5_theta1,label='Pendulum 5 bob 1')
plt.plot(t_eval,P5_theta2,label='Pendulum 5 bob 2')

#Formatting
plt.xlabel('Time')
plt.ylabel('Angular position')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

In [23]:
#Converting to cartesian

#First pendulum
P1_x1=l_1*np.sin(P1_theta1)
P1_y1=-l_1*np.cos(P1_theta1)

P1_x2=P1_x1+l_2*np.sin(P1_theta2)
P1_y2=P1_y1-l_2*np.cos(P1_theta2)


#Second pendulum
P2_x1=l_1*np.sin(P2_theta1)
P2_y1=-l_1*np.cos(P2_theta1)

P2_x2=P2_x1+l_2*np.sin(P2_theta2)
P2_y2=P2_y1-l_2*np.cos(P2_theta2)


#Third pendulum
P3_x1=l_1*np.sin(P3_theta1)
P3_y1=-l_1*np.cos(P3_theta1)

P3_x2=P3_x1+l_2*np.sin(P3_theta2)
P3_y2=P3_y1-l_2*np.cos(P3_theta2)


#Fourth pendulum
P4_x1=l_1*np.sin(P4_theta1)
P4_y1=-l_1*np.cos(P4_theta1)

P4_x2=P4_x1+l_2*np.sin(P4_theta2)
P4_y2=P4_y1-l_2*np.cos(P4_theta2)


#Fifth pendulum
P5_x1=l_1*np.sin(P5_theta1)
P5_y1=-l_1*np.cos(P5_theta1)

P5_x2=P5_x1+l_2*np.sin(P5_theta2)
P5_y2=P5_y1-l_2*np.cos(P5_theta2)

In [ ]:
#Animating the trajectory of the double pendulum


#Setting up the figure environment
fig, ax = plt.subplots()
ax.set_aspect('equal', 'box')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

#Colors for the 5 pendulums
colors = ['r', 'g', 'b', 'm', 'c']


# List of coordinate arrays for convenience
pendulums = [
    (P1_x1, P1_y1, P1_x2, P1_y2),
    (P2_x1, P2_y1, P2_x2, P2_y2),
    (P3_x1, P3_y1, P3_x2, P3_y2),
    (P4_x1, P4_y1, P4_x2, P4_y2),
    (P5_x1, P5_y1, P5_x2, P5_y2)
]

#Create line artists
lines = []
traces = []
trace_x = [[] for _ in range(5)]
trace_y = [[] for _ in range(5)]

for i in range(5):
    line, = ax.plot([], [], 'o-', lw=1.5, color=colors[i])
    trace, = ax.plot([], [], '-', lw=0.5, color=colors[i], alpha=0.6)
    lines.append(line)
    traces.append(trace)


#Add labels and title
ax.set_xlabel('X position (m)')
ax.set_ylabel('Y position (m)')
ax.set_title("Demonstration of the Chaotic Double Pendulum", fontsize=12, fontweight='bold')


#Initialization function
def init():
    for line, trace in zip(lines, traces):
        line.set_data([], [])
        trace.set_data([], [])
    return lines + traces

#Update function
def update(frame):
    for i, (x1, y1, x2, y2) in enumerate(pendulums):
        
        # update the rod
        lines[i].set_data([0, x1[frame], x2[frame]],
                          [0, y1[frame], y2[frame]])

        # update trace
        trace_x[i].append(x2[frame])
        trace_y[i].append(y2[frame])
        traces[i].set_data(trace_x[i], trace_y[i])

    return lines + traces

#Animation
ani = FuncAnimation(fig, update, frames=len(P1_x1),
                    init_func=init, blit=True, interval=20)



#Animation display using HTML5
HTML(ani.to_jshtml())